# Usage Examples

This document outlines the different functionalities available within the Box Embeddings package

### 0. Installing the package on your machine

*If you have the repo cloned*
```
pip install --editable . --user
```

### 1. Initialize a standard box tensor and check its parameters

To represent a Tensor as a Box, we use the class `BoxTensor`. The necessary parameter is `data` (a tensor)

In [59]:
from box_embeddings.parameterizations.box_tensor import *

# Let's create a toy example
data = torch.tensor([[[1, 2, 3], [3, 4, 6]],
          [[5, 6, 8], [7, 9, 5]]])
print(data.size())
box = BoxTensor(data)

torch.Size([2, 2, 3])


We can use several methods to look at the parameters of our box, such as

In [60]:
# Lower left coordinate
print(box.z)
# Top right coordinate
print(box.Z)
# Center coordinate
print(box.centre)


tensor([[1, 2, 3],
        [5, 6, 8]])
tensor([[3, 4, 6],
        [7, 9, 5]])
tensor([[2.0000, 3.0000, 4.5000],
        [6.0000, 7.5000, 6.5000]])


Let's broadcast our box to a new shape.. Broadcasting is often needed for different arithmetic operations. The function we
will use is `broadcast()`, and the required parameter is `target_shape=()`, which specify the new shape
for the box. This is very similar to `numpy.broadcast_to()`

In [74]:
data = torch.tensor([[[1, 2, 3], [3, 4, 6]],
          [[5, 6, 8], [7, 9, 5]]])
box = BoxTensor(data)
print('previous shape is:', box.box_shape)
box.broadcast(target_shape=(1, 2, 3))
print('after broadcasting:', box.box_shape)

previous shape is: (2, 3)
after broadcasting: (1, 2, 3)


In [ ]:
# TODO
from box_embeddings.parameterizations.delta_box_tensor import *
from box_embeddings.initializations.uniform_boxes import *

### 2. Box Volume

In [75]:
# TODO
# Currently no codes yet
from box_embeddings.modules.volume import *

### 3. Box Regularization

In [76]:
# TODO
# Currently no codes yet
from box_embeddings.modules.regularization import *

### 4. Box Intersection

In [1]:
from box_embeddings.modules.intersection import *

AllenNLP not available. Registrable won't work.


### 5. Box Pooling

In [78]:
from box_embeddings.modules.pooling import *
